In [2]:
from pymongo import MongoClient
from bson import ObjectId
import pandas as pd
from collections import defaultdict
import numpy as np
from pprint import pprint

In [3]:
client = MongoClient()
candidates = client.lbj["excerpt-2017-02-20"]

In [7]:
c = candidates.find({"$and": [{"language": "en"}, {"$nor": [
    {"experiences": {"$exists": False}},
    {"experiences": {"$size": 0}},
    {"experiences": {"$size": 1}}
]}]})

In [4]:
pipeline = [
            #{"$match": {"$and": [ 
            #    {"language": "en"},
            #    {"$nor": [
            #        {"experiences": {"$exists": False}},
            #        {"experiences": {"$size": 0}},
            #        {"experiences": {"$size": 1}}
            #    ]}]}},
            {"$unwind": {
                "path": "$experiences",
                "includeArrayIndex": "job_index"
                }
            },
            {"$project": {
                "industry": 1,
                "job_index": 1,
                "company_name": "$experiences.companyName",
                "place": "$experiences.place",
                "function": "$experiences.function",
                "start_date": "$experiences.startDate",
                "end_date": "$experiences.endDate"
                #"duration": "$experiences.duration"
                #"mission": "$experiences.missions"
                }}
            ]

In [5]:
agg = candidates.aggregate(pipeline)

In [6]:
def _get_element(doc, field_name):
    if field_name in doc:
        return doc[field_name]
    return np.nan

def make_dataframe(curr):
    data = defaultdict(list)

    for doc in curr:
        data['_id'].append(doc['_id'])
        data['function'].append(_get_element(doc, 'function'))
        data['company_name'].append(_get_element(doc,'company_name'))
        data['industry'].append(_get_element(doc,'industry'))
        data['job_index'].append(doc['job_index'])        
        data['place'].append(_get_element(doc, 'place'))
        data['start_date'].append(_get_element(doc, 'start_date'))
        data['end_date'].append(_get_element(doc, 'end_date'))

    return data

In [7]:
data = make_dataframe(agg)

In [8]:
df = pd.DataFrame(data)

In [9]:
df["_id"] = df["_id"].apply(str)

In [11]:
bad_ids = df[pd.isna(df["function"])]._id.unique()

In [12]:
df.drop(df[df._id.isin(bad_ids)].index, inplace=True)

In [13]:
df.to_pickle("/data/rali7/Tmp/solimanz/data/pickles/excerpt-2017-02-20.pkl")